In [1]:
import pandas as pd
import numpy as np

# ====== Parameters ======
C = 1.6875        # Semiannual coupon
F = 100           # Face value
T = 5             # Maturity in years
Price = 100.03125 # Market price
times = np.arange(0.5, T+0.1, 0.5)  # Semiannual periods
y0 = 0.1          # Initial guess for yield

# ====== Newton's Method Iterations ======
y = [y0]
P_list = []
f_list = []
fprime_list = []
y_next_list = []

for i in range(4):
    exp_terms = np.exp(-y[-1]*times)
    P = np.sum(C*exp_terms) + F*np.exp(-y[-1]*T)
    P_list.append(P)
    f = P - Price
    f_list.append(f)
    fprime = np.sum(-times*C*exp_terms) - T*F*np.exp(-y[-1]*T)
    fprime_list.append(fprime)
    y_next = y[-1] - f/fprime
    y_next_list.append(y_next)
    y.append(y_next)

# Include iteration 0
exp_terms0 = np.exp(-y[0]*times)
P0 = np.sum(C*exp_terms0) + F*np.exp(-y[0]*T)
f0 = P0 - Price
fprime0 = np.sum(-times*C*exp_terms0) - T*F*np.exp(-y[0]*T)

iterations_df = pd.DataFrame({
    'Iteration': [0,1,2,3,4],
    'y_n': y[:5],
    'P(y_n)': [P0]+P_list,
    'f(y_n)': [f0]+f_list,
    "f'(y_n)": [fprime0]+fprime_list,
    'y_{n+1}': [y_next_list[0]]+y_next_list
})

# ====== Duration and Convexity ======
yield_final = y[-1]
exp_terms_y = np.exp(-yield_final*times)
PV_coupons = C*exp_terms_y
tPV = times*PV_coupons
t2PV = (times**2)*PV_coupons
PV_face = F*np.exp(-yield_final*T)
tPV_face = T*PV_face
t2PV_face = T**2*PV_face

duration = (np.sum(tPV)+tPV_face)/Price
convexity = (np.sum(t2PV)+t2PV_face)/Price

dur_conv_df = pd.DataFrame({
    't': np.append(times, T),
    'PV': np.append(PV_coupons, PV_face),
    't*PV': np.append(tPV, tPV_face),
    't^2*PV': np.append(t2PV, t2PV_face)
})

summary_df = pd.DataFrame({
    'Price': [Price],
    'Yield (continuous)': [yield_final],
    'Duration': [duration],
    'Convexity': [convexity]
})

print(summary_df)

       Price  Yield (continuous)  Duration  Convexity
0  100.03125            0.033401  4.642735  22.573118


In [2]:
print('The yield after each iteration')
print(y_next_list)

The yield after each iteration
[0.021397014741595916, 0.03305732886033832, 0.03340081140223472, 0.033401098538030694]


In [3]:
print(iterations_df)

   Iteration       y_n      P(y_n)     f(y_n)     f'(y_n)   y_{n+1}
0          0  0.100000   73.603433 -26.427817 -336.219000  0.021397
1          1  0.021397   73.603433 -26.427817 -336.219000  0.021397
2          2  0.033057  105.772115   5.740865 -492.342255  0.033057
3          3  0.033401  100.191037   0.159787 -465.195486  0.033401
4          4  0.033401  100.031383   0.000133 -464.419239  0.033401
